In [1]:
from unittest.util import _count_diff_hashable
import numpy as np
import os
import collections
from os.path import dirname, abspath
from copy import deepcopy
from sacred import Experiment, SETTINGS
from sacred.observers import FileStorageObserver
from sacred.utils import apply_backspaces_and_linefeeds
import sys
import torch as th
from utils.logging import get_logger
import yaml

from run import run

from run import args_sanity_check
from types import SimpleNamespace as SN

from learners import REGISTRY as le_REGISTRY
from runners import REGISTRY as r_REGISTRY
from controllers import REGISTRY as mac_REGISTRY
from components.episode_buffer import ReplayBuffer
from components.episode_buffer import Prioritized_ReplayBuffer

from envs import REGISTRY as env_REGISTRY
from functools import partial

from components.episodic_memory_buffer import Episodic_memory_buffer
from components.transforms import OneHot


SETTINGS['CAPTURE_MODE'] = "fd" # set to "no" if you want to see stdout/stderr in console
logger = get_logger()

ex = Experiment('pymarl', interactive=True)
ex.logger = logger
ex.captured_out_filter = apply_backspaces_and_linefeeds

pygame 2.1.3 (SDL 2.0.22, Python 3.7.0)
Hello from the pygame community. https://www.pygame.org/contribute.html


[DEBUG 17:27:28] git.cmd Popen(['git', 'version'], cwd=/home/hellone/Project/ReinforcementLearning/EpisodicCuriosity/EMC/pymarl/src, universal_newlines=False, shell=None, istream=None)
[DEBUG 17:27:28] git.cmd Popen(['git', 'version'], cwd=/home/hellone/Project/ReinforcementLearning/EpisodicCuriosity/EMC/pymarl/src, universal_newlines=False, shell=None, istream=None)
[DEBUG 17:27:28] git.cmd Popen(['git', 'diff', '--cached', '--abbrev=40', '--full-index', '--raw'], cwd=/home/hellone/Project/ReinforcementLearning/EpisodicCuriosity/EMC, universal_newlines=False, shell=None, istream=None)
[DEBUG 17:27:28] git.cmd Popen(['git', 'diff', '--abbrev=40', '--full-index', '--raw'], cwd=/home/hellone/Project/ReinforcementLearning/EpisodicCuriosity/EMC, universal_newlines=False, shell=None, istream=None)
[DEBUG 17:27:28] git.cmd Popen(['git', 'cat-file', '--batch-check'], cwd=/home/hellone/Project/ReinforcementLearning/EpisodicCuriosity/EMC, universal_newlines=False, shell=None, istream=<valid str

In [2]:
__file__ = '/home/hellone/Project/ReinforcementLearning/EpisodicCuriosity/EMC/pymarl/src/'
results_path = os.path.join(dirname(dirname(abspath('/home/hellone/Project/ReinforcementLearning/EpisodicCuriosity/EMC/pymarl/src/'))), "results_debug")

In [3]:
argvs = ['debug.py', '--config=EMC_toygame', '--env-config=gridworld_reversed', 'with', 'env_args.map_name=reversed']
params = deepcopy(argvs)
params

['debug.py',
 '--config=EMC_toygame',
 '--env-config=gridworld_reversed',
 'with',
 'env_args.map_name=reversed']

In [4]:
def _get_config_env(params, arg_name, subfolder):
    config_name = None
    for _i, _v in enumerate(params):
        if _v.split("=")[0] == arg_name:
            config_name = _v.split("=")[1]
            del params[_i]
            break

    if config_name is not None:
        with open(os.path.join(os.path.dirname(__file__), "config", subfolder, "{}.yaml".format(config_name)), "r") as f:
            try:
                config_dict = yaml.load(f)
            except yaml.YAMLError as exc:
                assert False, "{}.yaml error: {}".format(config_name, exc)
        return config_dict

def _get_config_alg(params, arg_name, subfolder,map_name):
    config_name = None
    for _i, _v in enumerate(params):
        if _v.split("=")[0] == arg_name:
            config_name = _v.split("=")[1]
            del params[_i]
            break

    if map_name=="3s5z_vs_3s6z":
        config_name="EMC_sc2_3s5z_vs_3s6z"
    elif map_name=="6h_vs_8z":
        config_name="EMC_sc2_6h_vs_8z"
    elif map_name=="corridor":
        config_name="EMC_sc2_corridor"
    elif map_name=="origin":
        config_name="EMC_toygame"
    elif map_name=="reversed":
        config_name="EMC_toygame"


    

    if config_name is not None:
        with open(os.path.join(os.path.dirname(__file__), "config", subfolder, "{}.yaml".format(config_name)), "r") as f:
            try:
                config_dict = yaml.load(f)
            except yaml.YAMLError as exc:
                assert False, "{}.yaml error: {}".format(config_name, exc)
        return config_dict


def recursive_dict_update(d, u):
    for k, v in u.items():
        if isinstance(v, collections.Mapping):
            d[k] = recursive_dict_update(d.get(k, {}), v)
        else:
            d[k] = v
    return d


def config_copy(config):
    if isinstance(config, dict):
        return {k: config_copy(v) for k, v in config.items()}
    elif isinstance(config, list):
        return [config_copy(v) for v in config]
    else:
        return deepcopy(config)

In [5]:
with open(os.path.join(os.path.dirname(__file__), "config", "default.yaml"), "r") as f:
    try:
        config_dict = yaml.load(f)
    except yaml.YAMLError as exc:
        assert False, "default.yaml error: {}".format(exc)

env_config= _get_config_env(params, "--env-config", "envs")
config_dict = recursive_dict_update(config_dict, env_config)
map_name="3m"
for _i, _v in enumerate(params):
    if _v.split("=")[0] == "env_args.map_name":
        map_name = _v.split("=")[1]

print("Map Name:",map_name)
alg_config = _get_config_alg(params, "--config", "algs",map_name)
# config_dict = {**config_dict, **env_config, **alg_config}

config_dict = recursive_dict_update(config_dict, alg_config)

Map Name: reversed


/home/hellone/Project/ReinforcementLearning/EpisodicCuriosity/EMC/pymarl/src/:50: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working


In [6]:
config_dict['seed'] = 427095566

In [7]:
config = config_copy(config_dict)
np.random.seed(config["seed"])
th.manual_seed(config["seed"])
config['env_args']['seed'] = config["seed"]

In [8]:
args = SN(**config)
args.use_cuda = False
args.device = "cuda" if args.use_cuda else "cpu"
set_device = os.getenv('SET_DEVICE')
if args.use_cuda and set_device != '-1':
    if set_device is None:
        args.device = "cuda"
    else:
        args.device = f"cuda:{set_device}"
else:
    args.device = "cpu"

In [9]:
args.buffer_cpu_only = True

In [10]:
    # Init runner so we can get env info
runner = r_REGISTRY[args.runner](args=args, logger=logger)

# Set up schemes and groups here
env_info = runner.get_env_info()
args.episode_limit = env_info["episode_limit"]
args.n_agents = env_info["n_agents"]
args.n_actions = env_info["n_actions"]
args.state_shape = env_info["state_shape"]
args.unit_dim = env_info["unit_dim"]

# Default/Base scheme
scheme = {
    "state": {"vshape": env_info["state_shape"]},
    "obs": {"vshape": env_info["obs_shape"], "group": "agents"},
    "actions": {"vshape": (1,), "group": "agents", "dtype": th.long},
    "avail_actions": {"vshape": (env_info["n_actions"],), "group": "agents", "dtype": th.int},
    "reward": {"vshape": (1,)},
    "terminated": {"vshape": (1,), "dtype": th.uint8},
}
groups = {
    "agents": args.n_agents
}
preprocess = {
    "actions": ("actions_onehot", [OneHot(out_dim=args.n_actions)])
}

env_name = args.env

print(scheme)


{'state': {'vshape': 92}, 'obs': {'vshape': 46, 'group': 'agents'}, 'actions': {'vshape': (1,), 'group': 'agents', 'dtype': torch.int64}, 'avail_actions': {'vshape': (5,), 'group': 'agents', 'dtype': torch.int32}, 'reward': {'vshape': (1,)}, 'terminated': {'vshape': (1,), 'dtype': torch.uint8}}


In [11]:
buffer = Prioritized_ReplayBuffer(scheme, groups, args.buffer_size, env_info["episode_limit"] + 1,
                                        args.prioritized_buffer_alpha,
                                        preprocess=preprocess,
                                        device="cpu" if args.buffer_cpu_only else args.device)
ec_buffer=Episodic_memory_buffer(args,scheme)
mac = mac_REGISTRY[args.mac](buffer.scheme, groups, args)
# Give runner the scheme
# runner = episode
runner.setup(scheme=scheme, groups=groups, preprocess=preprocess, mac=mac)
# Learner
learner = le_REGISTRY[args.learner](mac, buffer.scheme, logger, args, groups=groups)

In [12]:
args.gener_goal_interval = 2

In [14]:
episode_batch = runner.run(test_mode=False)

KeyError: 'goal not found in transition or episode data'

In [ ]:
@ex.main
def my_main(_run, _config, _log):
    # Setting the random seed throughout the modules
    config = config_copy(_config)
    np.random.seed(config["seed"])
    th.manual_seed(config["seed"])
    config['env_args']['seed'] = config["seed"]

    # print(_run, '\n\n', config, '\n\n', _log)
    # run(_run, config, _log)

    print(config)
    # _config = args_sanity_check(config, _log)

    # args = SN(**_config)

ex.add_config(config_dict)
file_obs_path = os.path.join(results_path, "sacred")
ex.observers.append(FileStorageObserver.create(file_obs_path))
ex.run_commandline(params)

In [1]:
data = {
    "first": [1, 2 ,3],
    "second": [4, 5, 6],
}

In [5]:
data.items()

dict_items([('first', [1, 2, 3]), ('second', [4, 5, 6])])

In [4]:
for k , v in data.items():
# data.items()
    print(k, '\t', v)

first 	 [1, 2, 3]
second 	 [4, 5, 6]


In [ ]:
from envs import REGISTRY as env_REGISTRY
from functools import partial

In [51]:
class stu:
    def __init__(self) -> None:
        self.__mess = 6
        self.__name = 1

    @property
    def mess(self):
        return self.__mess
    
    @property
    def name(self):
        return self.__name

In [52]:
a = stu()

In [53]:
a._name

AttributeError: 'stu' object has no attribute '_name'

In [34]:
a.name

1

In [35]:
a._name = 2
a.name

2